In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, asc, desc, udf, from_unixtime, count
from datetime import datetime
from pyspark.sql.types import IntegerType, DateType

spark = SparkSession.\
        builder.\
        appName("dataframe").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        config("spark.jars", "/opt/data/sqlite-jdbc-3.36.0.3.jar").\
        getOrCreate()

22/05/16 05:41:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
ratings_df = spark.read.format("jdbc").\
    option("url", "jdbc:sqlite:/opt/data/movielens-small.db").\
    option("driver", "org.sqlite.JDBC").\
    option("dbtable", "ratings").\
    load()

In [3]:
def convert_to_week_number(timestamp):
    date_object = datetime.fromtimestamp(timestamp)

    return date_object.isocalendar()[1]

In [4]:
convertUDF = udf(lambda z: convert_to_week_number(z), IntegerType())

In [5]:
ratings_df = ratings_df.withColumn('week_number', convertUDF(ratings_df.timestamp))

In [9]:
grouped_ratings_df = ratings_df.groupBy("week_number", "userId").count().sort(desc("count")).show(5)

+-----------+------+-----+
|week_number|userId|count|
+-----------+------+-----+
|         43|   384| 1113|
|          4|   176|  972|
|         28|   614|  830|
|         34|   330|  711|
|         51|   202|  676|
+-----------+------+-----+
only showing top 5 rows

